In [2]:
!pip install pymongo
!pip install dnspython

import os
from pymongo import MongoClient

connection_string = os.environ.get('ConnectionStrings__photo-search')
client = MongoClient(connection_string)
db = client['photo-search']
# # List all collections
# collections = db.list_collection_names()

# print("Collections in tåe database:")
# for collection_name in collections:
#     print(collection_name)

collection = db['photos']
documents = collection.find()

print(documents[0]['PhotoSummaries']['llava-llama3']['Description'])
    

The image captures a vibrant scene from a concert at The O2 Arena in London, UK. The stage is bathed in bright lights against a backdrop of a large screen displaying an abstract design with hues of blue and red. 

On the stage, there are three musicians engrossed in their performance. Two of them are strumming guitars while the third one is energetically drumming on the drums. Their positions suggest they are in the middle of a song, possibly performing a popular hit.

The audience, visible in the foreground, appears to be thoroughly enjoying the concert. Many of them are holding up their phones, capturing moments from the performance and sharing it with others who couldn't make it to the venue.

The image is taken from an angle that provides a clear view of both the stage and the audience, creating a dynamic composition that encapsulates the excitement and energy of a live music concert.


In [3]:

import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
torch.cuda.device(0)

True
1
0
